In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime
from datetime import timedelta
from PIL import Image
from glob import glob


In [2]:
'''
Parametrização
'''

asset = 'BTOW3'
time_frame = '1d'
file_name = f'{asset}_{time_frame}.xlsx'
data_path = Path(f'./collected_data/{file_name}')
charts_path = Path(f'./generated_data/{asset}')


In [3]:
'''
Importa os dados do ativo e renomeia as colunas
'''

inplace = True
df_input = pd.read_excel(data_path)
df_input.dropna(inplace = inplace)
df_input.set_index('Data', inplace = inplace)
df_input.sort_index(axis = 0, inplace = inplace)
df_input.rename(inplace = inplace, columns = {
    'Abertura': 'Open',
    'Máxima': 'High',
    'Mínima': 'Low',
    'Fechamento': 'Close',
    'Volume Financeiro': 'Volume',
    'Média Móvel E [9]': 'MME009',
    'Média Móvel A [21]': 'MMA021',
    'Média Móvel A [200]': 'MMA200',
    'Média Móvel A [51]': 'MMA051',
    'Média Móvel A [80]': 'MMA080',
    'Média Móvel A [400]': 'MMA400',
    'Média Móvel E [400]': 'MME400'})

df_input.head()


,Open,High,Low,Close,Volume,MME009,MMA021,MMA200,MMA051,MMA080,MMA400,MME400
Data,,,,,,,,,,,,
2006-11-03,45.11,45.90,44.04,45.15,19193744,43.41,41.81,46.50,43.11,42.18,36.51,36.51
2006-11-06,45.20,46.72,44.20,46.50,32347216,44.03,41.99,46.49,43.19,42.23,36.57,36.56
2006-11-07,46.50,47.51,45.46,47.50,43484428,44.72,42.34,46.49,43.30,42.31,36.64,36.62
2006-11-08,47.39,47.90,46.14,47.75,52434539,45.33,42.76,46.50,43.40,42.41,36.71,36.67
2006-11-09,47.82,48.15,45.80,46.05,18398881,45.47,43.04,46.50,43.45,42.51,36.77,36.72


In [4]:
'''
Cria reordena as colunas dos indicadores em ordem alfabética
'''

df_candle = df_input[['Open', 'High', 'Low', 'Close', 'Volume']]
df_ind = df_input.reindex(sorted(df_input.columns[5:]), axis = 1)
df = pd.concat([df_candle, df_ind], axis=1)

# df.head()


In [5]:
'''
Verifica se o dataframe reordenado é igual ao original
'''

columns = list(df.columns)
eq = True
for col in columns:
   eq = eq*df[col].equals(df_input[col])
if eq:
    print("Data Frame válido.")
else:
    print("Data Frame inválido!!!!")
    

Data Frame válido.


In [6]:
'''
Aprender a gerar os gráficos para então iterar os intervalos de data
'''

def date_delta(start_date, delta): # Função sem uso por enquanto
    end_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days = delta)
    return end_date.strftime('%Y-%m-%d')


def gen_chart(df, date, trade_delta, min_profit, chart_delta, add, save_name):
    end_index = df.index.get_loc(date)
    start_index = end_index - chart_delta
    savefig = './generated_data/plot.png'
    
    start_trade = df['Open'][end_index + 1]
    end_trade = df['Close'][end_index + trade_delta]
    if end_trade/start_trade >= 1 + min_profit/100:
        trade_class = 'long'
    elif end_trade/start_trade <= 1 - min_profit/100:
        trade_class = 'short'
    else:
        trade_class = 'wait'

    df_plot = df.iloc[start_index:end_index,:]
    erase_color = 'black'

    mc = mpf.make_marketcolors(up = 'g',down = 'r',
                               wick = 'inherit',
                               edge = 'inherit',
                               volume = 'grey')
    s = mpf.make_mpf_style(marketcolors = mc,
                           facecolor = erase_color,
                           edgecolor = erase_color,
                           figcolor = erase_color)
    addplot = mpf.make_addplot(df_plot[add])
    plot = mpf.plot(df_plot, type = 'candle',
                             volume = True,
                             style = s,
                             addplot = addplot,
                             savefig = savefig)

    uncropped_plot = Image.open(savefig)
    box = (uncropped_plot.size[0]*0.202,
           uncropped_plot.size[1]*0.11,
           uncropped_plot.size[0]*0.88,
           uncropped_plot.size[1]*0.85)

    cropped_plot = uncropped_plot.crop(box).save(f'{save_name}_{trade_class}.png')
    return (start_trade, end_trade, trade_class)
    

In [9]:
'''
Chamamento das funções de geração dos gráficos
Implementar o loop for
'''

charts_path.mkdir(parents=True, exist_ok=True)
date_range = pd.date_range(start='2007-01-01', end='2020-06-30')
trade_delta = 3
min_profit = 1.5 # in %
chart_delta = 22
ind_columns = list(df.columns)[5:9]

count = 0
for date in date_range:
    try:
        file_date = date.strftime('%Y-%m-%d')
        save_name = charts_path/f'{asset}_{file_date}'
        out = gen_chart(df, date, trade_delta, min_profit, chart_delta, ind_columns, save_name)
        count += 1
        # print(f'{file_date} {out[0]} {out[1]} {out[2]}')
    except KeyError or IndexError:
        continue
print(f'{count} imagens geradas.')

3335 imagens geradas.
